In [ ]:
# yfinance not real time data causing issue 

In [1]:
import numpy as np
import pandas as pd
import time
import yfinance as yf
import ta

# Function to place a trade (Buy or Sell)
def place_trade(order_type, lot_size, price):
    # Placeholder function to simulate placing a trade
    # Replace this with actual code to place a real trade using a trading platform or API
    print(f"Placing a {order_type} order for {lot_size} lots at price {price}")

def cancel_pending():
    print("\nPending orders cancelled")

stock_name = 'RELIANCE.NS'
lot_size = 1  # Initial lot size
profit_factor = 2  # Factor to multiply lot size in case of a loss
martingale_limit = 5  # Number of consecutive losses allowed
oversold_rsi_threshold = 30
overbought_rsi_threshold = 70

# Wait until the start of the next minute
current_time = pd.Timestamp.now().time()
seconds_to_next_minute = 62 - current_time.second
time.sleep(seconds_to_next_minute)

consecutive_losses = 0

while True:
    # Interval required 1 minute
    data = yf.download(tickers=stock_name, period='1d', interval='1m')

    # Assuming 'data' contains the historical stock data fetched using yfinance
    previous_candle = data.iloc[-2]  # Get the row for the previous candle

    # Calculate RSI for the given period (14 in this case)
    data['rsi'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()

    if previous_candle['Close'] > previous_candle['Open']:
        print("Previous candle was bullish.")
        pre_bull_or_bear = 'bull'
        if data.iloc[-2]['rsi'] < oversold_rsi_threshold:
            # If RSI is less than the oversold threshold, Buy 1 lot / stock at the Close price of previous candle.
            place_trade("Buy", lot_size, previous_candle['Close'])
        else:
            print(f"RSI is {data.iloc[-2]['rsi']} not in oversold range. Skipping trade.")
    else:
        print("Previous candle was bearish.")
        pre_bull_or_bear = 'bear'
        if data.iloc[-2]['rsi'] > overbought_rsi_threshold:
            # If RSI is greater than the overbought threshold, Sell 1 lot / stock at the Close price of previous candle.
            place_trade("Sell", lot_size, previous_candle['Close'])
        else:
            print(f"RSI is {data.iloc[-2]['rsi']} not in overbought range. Skipping trade.")

    # Wait until the end of the current minute (59th second of the minute)
    current_time = pd.Timestamp.now().time()
    seconds_to_next_minute = 57 - current_time.second
    time.sleep(seconds_to_next_minute)
    cancel_pending()
    data = yf.download(tickers=stock_name, period='1d', interval='1m')
    current_candle = data.iloc[-1]

    if pre_bull_or_bear == 'bull':
        # At the end of the candle, check if the position is profitable
        if previous_candle['Close'] < current_candle['Close']:
            # The position was profitable, reset the consecutive losses counter
            print("Position was profitable. Keeping lot size at 1.")
            consecutive_losses = 0
        else:
            # The position was a loss
            consecutive_losses += 1
            if consecutive_losses >= martingale_limit:
                print(f"Reached Martingale Limit of {martingale_limit} consecutive losses. Stopping.")
                break
            else:
                # Multiply lot size by profit_factor
                print("Position was a loss. Multiplying lot size by 2.")
                lot_size *= profit_factor
    else:
        # At the end of the candle, check if the position is profitable
        if previous_candle['Close'] > current_candle['Close']:
            # The position was profitable, reset the consecutive losses counter
            print("Position was profitable. Keeping lot size at 1.")
            consecutive_losses = 0
        else:
            # The position was a loss
            consecutive_losses += 1
            if consecutive_losses >= martingale_limit:
                print(f"Reached Martingale Limit of {martingale_limit} consecutive losses. Stopping.")
                break
            else:
                # Multiply lot size by profit_factor
                print("Position was a loss. Multiplying lot size by 2.")
                lot_size *= profit_factor

    # Wait until the start of the next minute
    current_time = pd.Timestamp.now().time()
    seconds_to_next_minute = 62 - current_time.second
    time.sleep(seconds_to_next_minute)


[*********************100%***********************]  1 of 1 completed
Previous candle was bullish.
RSI is 35.04546591814639 not in oversold range. Skipping trade.

Pending orders cancelled
[*********************100%***********************]  1 of 1 completed
Position was profitable. Keeping lot size at 1.
[*********************100%***********************]  1 of 1 completed
Previous candle was bullish.
RSI is 35.04546591814639 not in oversold range. Skipping trade.

Pending orders cancelled
[*********************100%***********************]  1 of 1 completed
Position was profitable. Keeping lot size at 1.
[*********************100%***********************]  1 of 1 completed
Previous candle was bullish.
RSI is 35.04546591814639 not in oversold range. Skipping trade.

Pending orders cancelled
[*********************100%***********************]  1 of 1 completed
Position was profitable. Keeping lot size at 1.
[*********************100%***********************]  1 of 1 completed
Previous candle 